In [ ]:
# Based on 7700 distinct labels (700 labels and 10 synonyms per label retrieved) we would get approx. 30 Mio. possible activity label pair combinations. 
# Therefore strucuture approach 'semantic search pair sampling' based on idea from Thakur et al. 2021 to retrieve high scored synonyms onyl to reduce computational overhead.   

In [1]:
from torch.utils.data import DataLoader
from sentence_transformers import models, losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from datetime import datetime
import logging
import csv
import torch
import tqdm
import sys
import math
import gzip
import os

In [2]:
import pandas as pd
path = '/home/jupyter-sturm/Archive/01 Datasets/labels_BPIC12_aug.csv'
csv = pd.read_csv(path, sep=";")

In [8]:
sentences = list(csv['augmented_labels'])

In [9]:
#### Top k similar sentences to be retrieved ####
#### Larger the k, bigger the silver dataset ####
top_k = 10
logging.info("Step 2.1: Generate STSbenchmark (silver dataset) using pretrained SBERT \
    model and top-{} semantic search combinations".format(top_k))


silver_data = []
#sentences = set()

#for sample in gold_samples:
 #   sentences.update(sample.texts)

#sentences = list(sentences) # unique sentences
sent2idx = {sentence: idx for idx, sentence in enumerate(sentences)} # storing id and sentence in dictionary
#duplicates = set((sent2idx[data.texts[0]], sent2idx[data.texts[1]]) for data in gold_samples) # not to include gold pairs of sentences again

In [ ]:
print(sent2idx)

In [12]:
# For simplicity we use a pretrained model
batch_size = 16
num_epochs = 1
max_seq_length = 128

semantic_encoder_path = '/home/jupyter-sturm/output/bi-encoder/qqp_cross_domain_bert-base-uncased-2022-03-17_15-25-31'

#semantic_model_name = 'paraphrase-MiniLM-L6-v2'
semantic_search_model = SentenceTransformer(semantic_encoder_path)
logging.info("Encoding unique sentences with semantic search model: {}".format(semantic_search_model))

In [13]:
# encoding all unique sentences present in the training dataset using semantic model
embeddings = semantic_search_model.encode(sentences, batch_size=batch_size, convert_to_tensor=True)

#logging.info("Retrieve top-{} with semantic search model: {}".format(top_k, semantic_model_name))

In [14]:
# retrieving top-k sentences given a sentence from the dataset
#progress = tqdm.tqdm(unit="docs", total=len(sent2idx))
for idx in range(len(sentences)):
    sentence_embedding = embeddings[idx]
    cos_scores = util.cos_sim(sentence_embedding, embeddings)[0]
    cos_scores = cos_scores.cpu()
    #progress.update(1)

    #torch.topk to find the highest 10 scores
    top_results = torch.topk(cos_scores, k=top_k+1)
    
    for score, iid in zip(top_results[0], top_results[1]):
        if iid != idx: #and (iid, idx) not in duplicates:
            silver_data.append((sentences[idx], sentences[iid]))
            #duplicates.add((idx,iid))

#progress.reset()
#progress.close()

In [15]:
print(silver_data[2])

('diligence accepted', 'diligence finalized')
